## Dependencies

In [1]:
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
model_path = '/kaggle/input/106-googleq-a-train-bert-base-spatial-dropout/model.h5'
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
TOKENIZER_PATH = BERT_PATH+'/assets/vocab.txt'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(TOKENIZER_PATH, True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

bert_layer = KerasLayer(BERT_PATH, trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

x = SpatialDropout1D(0.2)(sequence_output)
x = GlobalAveragePooling1D()(x)
# x = Dropout(0.2)(x)
output = Dense(N_CLASS, kernel_initializer='glorot_uniform', activation="sigmoid", name="output")(x)

model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

# Make predictions

In [8]:
Y_test = model.predict(X_test)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.933097,0.513932,0.489267,0.380414,0.401237,0.472523,0.668069,0.596335,0.485526,...,0.891925,0.881086,0.515271,0.965868,0.927557,0.758678,0.061448,0.054358,0.797139,0.902886
1,46,0.926919,0.467885,0.008221,0.730224,0.777647,0.911825,0.563201,0.484499,0.079934,...,0.702236,0.959442,0.671250,0.974886,0.986587,0.858029,0.958467,0.100822,0.081397,0.870129
2,70,0.922352,0.578618,0.052606,0.744132,0.861608,0.930759,0.600732,0.504116,0.104582,...,0.854975,0.913238,0.636358,0.976907,0.956580,0.845711,0.082557,0.056153,0.828538,0.897354
3,132,0.872974,0.418244,0.013476,0.630963,0.755827,0.903195,0.549285,0.492521,0.075135,...,0.700401,0.926198,0.657604,0.966845,0.980358,0.864168,0.880933,0.153274,0.250928,0.867346
4,200,0.917592,0.366024,0.101930,0.643844,0.692052,0.754806,0.606683,0.529184,0.120367,...,0.696206,0.833338,0.562132,0.962186,0.943541,0.764456,0.195968,0.131209,0.479427,0.897769


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.902307,0.548637,0.047588,0.697402,0.764981,0.844138,0.575805,0.489398,0.206576,...,0.797931,0.914307,0.667236,0.963187,0.969392,0.835410,0.543738,0.124459,0.466645,0.904459
std,2812.670060,0.043839,0.120544,0.082618,0.122790,0.116037,0.140535,0.054165,0.082105,0.175461,...,0.080144,0.031280,0.053034,0.011756,0.014741,0.048082,0.347604,0.063660,0.288287,0.025443
min,39.000000,0.717862,0.299100,0.003470,0.218055,0.193474,0.188700,0.446886,0.308195,0.004791,...,0.584509,0.789391,0.459813,0.925326,0.898422,0.678235,0.005641,0.010554,0.018287,0.814103
25%,2572.000000,0.876080,0.446509,0.008881,0.630898,0.725271,0.812132,0.536925,0.432359,0.079763,...,0.737290,0.897790,0.635244,0.955698,0.962420,0.805341,0.170903,0.081112,0.208431,0.888805
50%,5093.000000,0.905143,0.527279,0.015246,0.705466,0.781815,0.898566,0.569257,0.469925,0.143543,...,0.791400,0.917297,0.668324,0.964846,0.972838,0.844078,0.623529,0.115747,0.444202,0.905661
75%,7482.000000,0.937008,0.636964,0.041709,0.777974,0.829567,0.933986,0.611043,0.534207,0.279164,...,0.867434,0.936042,0.697803,0.971641,0.980284,0.870322,0.867422,0.165890,0.713419,0.922995
max,9640.000000,0.978755,0.842806,0.637628,0.973849,0.975836,0.986819,0.740618,0.745324,0.763130,...,0.939896,0.976982,0.808773,0.989670,0.992231,0.941394,0.981617,0.339030,0.986525,0.964068
